In [10]:
import re
import weaviate
import numpy as np
import string
import numpy as np
import re
import json


auth_config = weaviate.AuthApiKey(api_key="cSS9XqOYrQ47rPmzCogrKYYm6rDhaZ7DJdCX")

client = weaviate.Client(
    url="https://test-s3sbv82a.weaviate.network",
    auth_client_secret=auth_config
)
paths = ["test-dataset-1.md", "test-dataset-2.md"]

# 腾讯
from gensim.models import KeyedVectors
file = 'word/tencent-ailab-embedding-zh-d100-v0.2.0-s.txt'
wv_from_text = KeyedVectors.load_word2vec_format(file, binary=False)

# 清除类
client.schema.delete_class("Article")

# 创建类
article_class = {
    "class": "Article",
    "description": "A class to represent articles",
    "properties": [
        {
            "name": "title",
            "description": "The title of the article",
            "dataType": ["text"]
        },
        {
            "name": "author",
            "description": "The author of the article",
            "dataType": ["text"]
        },
        {
            "name": "content",
            "description": "The content of the article",
            "dataType": ["text"]
        }
    ]
}

# 在Weaviate中添加类定义
client.schema.create_class(article_class)
# # get the schema
# schema = client.schema.get()

# # print the schema
# print(json.dumps(schema, indent=4))

In [15]:
# Initialize counter and interval for displaying progress
counter = 0
interval = 200

def process_data(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()
    title_pattern = r"# (.*?)\n"
    author_pattern = r">\s*(.*?)\n"
    content_pattern = r"(?:>\s*.*?\n)([\s\S]*)"
    title = re.search(title_pattern, content).group(1)
    author = re.search(author_pattern, content).group(1)
    article_content = re.search(content_pattern, content).group(1)
    punctuation = ' '
    article_content = re.sub(f"[{re.escape(punctuation)}]", "", article_content)
    article_content = article_content.replace("\n", "")
    pattern = r'\*\*(.*?)\*\*|。|；'
    segments = re.split(pattern, article_content)
    # 过滤空字符串和None
    segments = [seg.strip() for seg in segments if seg and seg.strip() not in ['##']]
    return title, author, segments

# Function to add articles to batch
def add_article_to_batch(batch, title, author, segment, wv_from_text):
    global counter
    sentence_vector = [wv_from_text[word] for word in segment if word in wv_from_text.key_to_index]
    if sentence_vector:
        sentence_vector_mean = np.mean(sentence_vector, axis=0).tolist()
    else:
        sentence_vector_mean = []
    properties = {
        "title": title,
        "author": author,
        "content": segment.strip()
    }
    batch.add_data_object(properties, 'Article', vector=sentence_vector_mean)
    
    # Calculate and display progress
    counter += 1
    if counter % interval == 0:
        print(f'Imported {counter} segments...')


# Read your data (assuming you have a way to get 'title', 'author', and 'segments')
for path in paths:
    title, author, segments = process_data(path)
    # Configure batch once
    client.batch.configure(batch_size=100)

    # Use the batch
    with client.batch as batch:
        for segment in segments:  # assuming 'segments' is a list of text segments from your article
            add_article_to_batch(batch, title, author, segment, wv_from_text)

    print(f'Finished importing {counter} segments.')

def search_Query(keyword):
    keyword = keyword
    keyword_vector = [wv_from_text[word] for word in keyword if word in wv_from_text.key_to_index]
    keyword_vector_mean = np.mean(keyword_vector, axis=0).tolist()

    result = (
        client.query
        .get("Article", ["title", "content","author"])
        .with_hybrid(keyword, alpha=0.5,properties=["content"],vector=keyword_vector_mean)
        .with_limit(5)
        .do()
    )
    print(json.dumps(result, indent=4, ensure_ascii=False))

search_Query('毫无疑问')

Finished importing 45 segments.
Imported 200 segments...
Finished importing 364 segments.
{
    "data": {
        "Get": {
            "Article": [
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "传统财务资产的特征：固定资产会折旧，部分流动资产（如库存和应收账款）会减值，金融性资产会根据市场需求上下波动，货币会贬值，以知识产权为主的无形资产会因技术进步而贬值",
                    "title": "科技投行｜重新认识和界定技术资产"
                },
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "技术资产与财务资产的区别",
                    "title": "科技投行｜重新认识和界定技术资产"
                },
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "就是由专业人才来运作国家技术资本投资基金，通过信息披露与第三方专业机构的考评与监督制度，提高投资效率，减少权力腐败与行政低效",
                    "title": "创新驱动引擎：原理、模式与布局"
                },
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "研发投入主要是人力资源成本和固定资产投入",
                    "title": "科技投行｜重新认识和界定技术资产"
                },
                {
   

In [16]:
search_Query('毫无疑问')

{
    "data": {
        "Get": {
            "Article": [
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "毫无疑问，技术资产是未来支撑具有硬核科创属性和持续高效发展的上市企业市值和商誉的核心资产",
                    "title": "科技投行｜重新认识和界定技术资产"
                },
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "无畏的执行者则是通过不懈的努力将愿景变成现实",
                    "title": "创新驱动引擎：原理、模式与布局"
                },
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "约翰·亨尼斯表示创新需要几类人：有远见的预言者、探索者、无畏的执行者。",
                    "title": "创新驱动引擎：原理、模式与布局"
                },
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "这种生态系统形成了彼此依赖、互补共生、协作繁荣的利益格局，谁也无法卡谁的脖子",
                    "title": "创新驱动引擎：原理、模式与布局"
                },
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "这是一个很容易被忽视的观察维度",
                    "title": "创新驱动引擎：原

In [12]:
title

'科技投行｜重新认识和界定技术资产'

In [6]:
result = (
    client.query
    .get("Article", ["title", "content","author"])
    .with_limit(50)
    .do()
)
print(json.dumps(result, indent=4, ensure_ascii=False))


{
    "data": {
        "Get": {
            "Article": [
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "对投资人来讲技术资产质量决定了投资风险与回报",
                    "title": "科技投行｜重新认识和界定技术资产"
                },
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "技术资产是科创企业的核心竞争力资产",
                    "title": "科技投行｜重新认识和界定技术资产"
                },
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "六大战略性产业国家级专精特新成为科创版上市公司的主要构成市值最高的头部公司中行业标杆企业相继涌现科创板上市企业大多轻资产、高估值共26家企业市值超过400亿元科创板整体PB和PE估值水平高于A股主板这表科创板大力支持极具潜力、高估值的科技创新企业发展这也说明科创板对未实现盈利的高新技术企业容纳度更高是高价值技术资产得到投资者认可的表现毫无疑问技术资产是未来支撑具有硬核科创属性和持续高效发展的上市企业市值和商誉的核心资产但是对于技术资产的定义业界和学术界却一直存在争议我们认为面对战略新兴产业的大发展结合全球的实践经验有必要对技术资产做出一个清晰量化的、可落地服务于投资、交易、并购估值和评价的务实定义##",
                    "title": "科技投行｜重新认识和界定技术资产"
                },
                {
                    "author": "杨川，新诤信知识产权",
                    "content": "科技类企业指的是竞争优势与